In [7]:
import numpy as np
import pandas as pd
import json
import os
from tabulate import tabulate

import matplotlib.pyplot as plt

In [36]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\Qiskit-Vs-PL_pauli\PL\Files"
#folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\Qiskit-Vs-PL_pauli\Qiskit\Files"
dataframes = {}
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
folders

['AHO', 'DW', 'QHO']

In [37]:
converged_runs = []
mean_iters = []
mean_evals = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
mean_e = []
delta_mean_e = []
exact_e = []
time_taken = []

num_vqe = []
shots_list = []
maxiter = []

cutoff_list = [2,4,8]
potentials = []
cutoffs = []


for folder in folders:
    for cutoff in cutoff_list:

        d_path = os.path.join(folder_path, folder, f'{folder}_{cutoff}.json')
    
        try:
            with open(d_path, 'r') as json_file:
                data = json.load(json_file)
        

            converged_indices = [i for i, success in enumerate(data['success']) if success]

            potentials.append(folder)
            cutoffs.append(cutoff)

            num_vqe.append(data['num_VQE'])
            shots_list.append(data['shots'])

            optimzer = data['Optimizer']
            maxiter.append(optimzer['maxiter'])
            
            exact_min_e = np.min(data['exact_eigenvalues'])
            exact_e.append(exact_min_e)

            converged_runs.append(sum(data['success']))

            #iters = [data['num_iters'][i] for i in converged_indices]
            #mean_iters.append(int(np.round(np.mean(iters))))
            mean_iters.append(int(np.round(np.mean(data['num_iters']))))

            #evals = [data['num_evaluations'][i] for i in converged_indices]
            #mean_evals.append(int(np.round(np.mean(evals))))
            mean_evals.append(int(np.round(np.mean(data['num_evaluations']))))

            #results = [data['results'][i] for i in converged_indices]
            results = data['results']
            min_e.append(np.min(results))
            delta_min_e.append(abs(exact_min_e - np.min(results)))
            median_e.append(np.median(results))
            delta_median_e.append(abs(exact_min_e - np.median(results)))
            mean_e.append(np.mean(results))
            delta_mean_e.append(abs(exact_min_e - np.mean(results)))

            #run_times = [data['run_times'][i] for i in converged_indices]
            #time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
            time_series = np.array([pd.Timedelta(t).total_seconds() for t in data['run_times']])
            mean_vqe_time = np.mean(time_series)
            #mean_vqe_time = pd.Timedelta(seconds=np.mean(time_series))
            time_taken.append(np.mean(mean_vqe_time))
            #time_taken.append(data['parallel_run_time'])

        except Exception as e:
            print("Unable to open file...")

In [35]:
qiskit_df = pd.DataFrame({
    'potential': potentials,
    'cutoff': cutoffs,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'Mean Iter': mean_iters,
    #'Mean evals': mean_evals,
    'Mean VQE time (s)': [f"{t:.2f}" for t in time_taken],
    #'E_VQE': [f"{num:.5e}" for num in median_e],
    #'Exact': [f"{num:.5e}" for num in exact_e],
    #'Delta': [f"{num:.5e}" for num in delta_median_e],
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    #'VQE mean E': [f"{num:.5e}" for num in mean_e],
    'Delta mean E': [f"{num:.5e}" for num in delta_mean_e]
    
})

qiskit_df

,potential,cutoff,Converged Runs,Mean VQE time (s),Delta min E,Delta median E,Delta mean E
0,AHO,2,100/100,13.79,1.11022e-16,1.11022e-16,2.50000e-06
1,AHO,4,0/100,5448.59,2.38402e-01,1.95664e-01,1.95610e-01
2,AHO,8,0/100,12555.78,5.53378e+00,4.60396e+00,4.60356e+00
3,DW,2,100/100,13.85,0.00000e+00,0.00000e+00,2.69031e-05
4,DW,4,0/100,11149.29,2.02432e-01,1.42673e-01,1.47833e-01
5,DW,8,0/100,31023.14,1.04995e+00,7.56279e-01,7.67317e-01
6,QHO,2,100/100,12.65,0.00000e+00,0.00000e+00,0.00000e+00
7,QHO,4,100/100,27.56,0.00000e+00,1.00000e-04,1.72000e-04
8,QHO,8,100/100,35.41,0.00000e+00,2.00000e-04,3.02000e-04


In [38]:
pl_df = pd.DataFrame({
    'potential': potentials,
    'cutoff': cutoffs,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'Mean Iter': mean_iters,
    #'Mean evals': mean_evals,
    'Mean VQE time (s)': [f"{t:.2f}" for t in time_taken],
    #'E_VQE': [f"{num:.5e}" for num in median_e],
    #'Exact': [f"{num:.5e}" for num in exact_e],
    #'Delta': [f"{num:.5e}" for num in delta_median_e],
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    #'VQE mean E': [f"{num:.5e}" for num in mean_e],
    'Delta mean E': [f"{num:.5e}" for num in delta_mean_e]
    
})

pl_df

,potential,cutoff,Converged Runs,Mean VQE time (s),Delta min E,Delta median E,Delta mean E
0,AHO,2,100/100,2.32,1.11022e-16,1.11022e-16,3.00000e-05
1,AHO,4,100/100,17.17,1.23665e-01,2.57614e-02,2.26079e-02
2,AHO,8,100/100,217.94,5.48290e+00,2.99199e+00,3.00462e+00
3,DW,2,100/100,3.95,1.11022e-16,1.11022e-16,5.55929e-05
4,DW,4,100/100,35.78,1.02416e-01,3.01967e-02,2.96203e-02
5,DW,8,100/100,377.82,8.08536e-01,4.76762e-01,4.80381e-01
6,QHO,2,100/100,2.06,0.00000e+00,0.00000e+00,1.00000e-05
7,QHO,4,100/100,6.66,1.11022e-16,2.00000e-04,2.65000e-04
8,QHO,8,100/100,10.18,0.00000e+00,4.00000e-04,4.72000e-04


In [42]:
merged = pl_df.merge(qiskit_df.reset_index(), on=['potential', 'cutoff'], suffixes=('_pl', '_qk')).drop(columns=['index'])
merged


,potential,cutoff,Converged Runs_pl,Mean VQE time (s)_pl,Delta min E_pl,Delta median E_pl,Delta mean E_pl,Converged Runs_qk,Mean VQE time (s)_qk,Delta min E_qk,Delta median E_qk,Delta mean E_qk
0,AHO,2,100/100,2.32,1.11022e-16,1.11022e-16,3.00000e-05,100/100,13.79,1.11022e-16,1.11022e-16,2.50000e-06
1,AHO,4,100/100,17.17,1.23665e-01,2.57614e-02,2.26079e-02,0/100,5448.59,2.38402e-01,1.95664e-01,1.95610e-01
2,AHO,8,100/100,217.94,5.48290e+00,2.99199e+00,3.00462e+00,0/100,12555.78,5.53378e+00,4.60396e+00,4.60356e+00
3,DW,2,100/100,3.95,1.11022e-16,1.11022e-16,5.55929e-05,100/100,13.85,0.00000e+00,0.00000e+00,2.69031e-05
4,DW,4,100/100,35.78,1.02416e-01,3.01967e-02,2.96203e-02,0/100,11149.29,2.02432e-01,1.42673e-01,1.47833e-01
5,DW,8,100/100,377.82,8.08536e-01,4.76762e-01,4.80381e-01,0/100,31023.14,1.04995e+00,7.56279e-01,7.67317e-01
6,QHO,2,100/100,2.06,0.00000e+00,0.00000e+00,1.00000e-05,100/100,12.65,0.00000e+00,0.00000e+00,0.00000e+00
7,QHO,4,100/100,6.66,1.11022e-16,2.00000e-04,2.65000e-04,100/100,27.56,0.00000e+00,1.00000e-04,1.72000e-04
8,QHO,8,100/100,10.18,0.00000e+00,4.00000e-04,4.72000e-04,100/100,35.41,0.00000e+00,2.00000e-04,3.02000e-04


In [51]:
exclude = ['potential', 'cutoff', 'Converged Runs_pl','Converged Runs_qk']
cols_to_float = [c for c in merged.columns if c not in exclude]

merged[cols_to_float] = merged[cols_to_float].astype(float)


In [63]:
def compare_metrics(row, pairs, tol=1e-15):
    styles = []
    for pl_col, qk_col in pairs:
        pl_val, qk_val = row[pl_col], row[qk_col]

        if abs(pl_val - qk_val) <= tol:
            styles += ["background-color: #ffffcc", "background-color: #ffffcc"]
        elif pl_val < qk_val:
            styles += ["background-color: green", ""]
        else:
            styles += ["", "background-color: green"]
    return styles


In [66]:
metric_pairs = [("Mean VQE time (s)_pl", "Mean VQE time (s)_qk"), ("Delta min E_pl","Delta min E_qk"), ("Delta median E_pl","Delta median E_qk"), ("Delta mean E_pl","Delta mean E_qk")]
merged.style.apply(lambda r: compare_metrics(r, metric_pairs),
                        subset=[c for pair in metric_pairs for c in pair],
                        axis=1)

,potential,cutoff,Converged Runs_pl,Mean VQE time (s)_pl,Delta min E_pl,Delta median E_pl,Delta mean E_pl,Converged Runs_qk,Mean VQE time (s)_qk,Delta min E_qk,Delta median E_qk,Delta mean E_qk
0,AHO,2,100/100,2.320000,0.000000,0.000000,0.000030,100/100,13.790000,0.000000,0.000000,0.000003
1,AHO,4,100/100,17.170000,0.123665,0.025761,0.022608,0/100,5448.590000,0.238402,0.195664,0.195610
2,AHO,8,100/100,217.940000,5.482900,2.991990,3.004620,0/100,12555.780000,5.533780,4.603960,4.603560
3,DW,2,100/100,3.950000,0.000000,0.000000,0.000056,100/100,13.850000,0.000000,0.000000,0.000027
4,DW,4,100/100,35.780000,0.102416,0.030197,0.029620,0/100,11149.290000,0.202432,0.142673,0.147833
5,DW,8,100/100,377.820000,0.808536,0.476762,0.480381,0/100,31023.140000,1.049950,0.756279,0.767317
6,QHO,2,100/100,2.060000,0.000000,0.000000,0.000010,100/100,12.650000,0.000000,0.000000,0.000000
7,QHO,4,100/100,6.660000,0.000000,0.000200,0.000265,100/100,27.560000,0.000000,0.000100,0.000172
8,QHO,8,100/100,10.180000,0.000000,0.000400,0.000472,100/100,35.410000,0.000000,0.000200,0.000302
